## Flavor Network

We plot the backbone of the flavor network,  presented in 
 [Nature](https://www.nature.com/articles/srep00196)
 
  
The file  `'flavor-network.json'` 
was set up from the files provided by the authors, here: [https://www.nature.com/articles/srep00196](https://www.nature.com/articles/srep00196), 
and here [http://yongyeol.com/pub/](http://yongyeol.com/pub/).

The json file is a dict with three keys: `'name'`, `'nodes'` and `'edges'`. 
We also saved in this file the position of each node,  assigned by the `fdp` `pygraphviz` layout.

The nodes of the network are the ingredients and node color corresponds to the  food category.
Two ingredients are connected if they share a  number of flavor compounds.   
 While hovering the mouse over a node  the ingredient name and its prevalence is displayed.
Since for some categories the edges are cluttered we provide the posibility to plot its 
corresponding subgraph as a Python igraph with the Kamada-Kaway 3d layout.

In [1]:
import numpy as np
import json
import igraph as ig
import copy
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

### The backbone of the flavor network

In [2]:
def Plotly_edges(E, node_position):

    Xe=[]
    Ye=[]
    for e in E:
        Xe+=[node_position[e[0]][0], node_position[e[1]][0], None]# x coordinates of the nodes defining the edge e
        Ye+=[node_position[e[0]][1], node_position[e[1]][1], None]# y 
    return dict(type='scatter', 
                x=Xe,
                y=Ye, 
                mode='lines',
                line=dict(color='rgb(160,160,160)', width=0.5),
                hoverinfo='none',
                name='edge'
               )  

def layout2d(V, node_position):
    axis=dict(showbackground=False,# 
              showline=False,  
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='' 
             )

    layout = dict(title="Backbone of the Flavor Network", 
                  font=dict(family='Balto', color='rgb(5,5,5)'),
                  width=900,
                  height=800,
                  showlegend=True,
                  xaxis=dict(axis),
                  yaxis=dict(axis), 
         
                  margin=dict(t=100),
                  hovermode='closest',
                  legend=dict(x=1.03, y=0.9),
                  paper_bgcolor='rgb(235,235,235)',
                  annotations=[dict(showarrow=False, 
                                    text="Data source: <a href='https://www.nature.com/articles/srep00196'> [1]</a>",  
                                    xref='paper',     
                                    yref='paper',     
                                    x=0,  
                                    y=-0.05,  
                                    xanchor='left',    
                                    yanchor='top',  
                                    font=dict(family='Balto',
                                    size=14)     
                                )
                            ]
                 )

    anno_text=['flower', 'red_bean', 'onion', 'beef', 'fish', 'cottage_cheese', 'wine',
               'olive', 'citrus', 'fennel', 'pepper', 'basil', 'white_bread', 'mushroom']


    x_annotext=[node_position[V.index(txt)][0] for txt in anno_text]
    y_annotext=[node_position[V.index(txt)][1] for txt in anno_text]
    x_anchor=['right', 'right', 'right', 'center', 'center', 'center', 'left', 
                        'center', 'left', 'left', 'right', 'right', 'left', 'left']
    y_anchor=['bottom', 'center', 'center', 'bottom', 'top', 'bottom', 'center',
                        'bottom', 'center', 'center', 'center','bottom',  'center', 'center']
    step=4#a value we add/subtract to x_annotext, y_annotext in order to avoid overlaying the annotext and the node dot
    for k in range(len(x_anchor)):
        if x_anchor[k]=='right': x_annotext[k]-=step
        elif x_anchor[k]=='left': x_annotext[k]+=step
        else:pass
    for k in range(len(y_anchor)):
        if y_anchor[k]=='top': y_annotext[k]-=step
        elif y_anchor[k]=='bottom': y_annotext[k]+=step 
        else: pass    

    layout['annotations']=layout['annotations']+[dict(showarrow=False,
                                                  text=anno_text[k],
                                                  xref='x',
                                                  yref='y',
                                                  x=x_annotext[k],
                                                  y=y_annotext[k],
                                                  font=dict(family='Balto', size=11, color='rgb(5,5,5)'),
                                                  xanchor=x_anchor[k],   
                                                  yanchor=y_anchor[k]
                                                 ) for k in range(len(anno_text))
                                            ]
    return layout




In [3]:
####  Process data read from the json file
with open("Data/flavor-network.json") as json_file:
    network = json.load(json_file)
#print (network.keys())

V=[net['ingredient'] for net in network['nodes']]#list of nodes
N=len(V)
d_index_ingr=dict(zip(range(N), V))
ingr_preval=[net['prevalence'] for net in network['nodes']]# list of prevalence values for each ingredient(node)
ingr_category={net['ingredient']: net['category'] for net in network['nodes']}# dict ingredient: category
categories=list(set(ingr_category.values()))# list of categories
n_cat=len(categories)
cat_indices=[[V.index(v)  for v in V if ingr_category[v]==categories[k]] for k in range(n_cat)]
node_position=[net['position'] for net in network['nodes']]#node position assigned via pygraphviz fdp layout

Edges=[tuple(e['edge']) for e in network['edges']]
E=[(V.index(e[0]), V.index(e[1]))  for e in Edges]# edges represented as tuple of linked node indices


colors={'fruit': '#FFF700',
        'herb':'rgb(102,194,163)',
        'cereal/crop':'rgb(247,231,180)' ,
        'flower': '#f500f5',
        'spice':  '#ffa517', 
        'alcoholic beverage':'#840123',
        'meat': '#d21717',
        'animal product':'#d2823b',
        'vegetable': '#009b00', 
        'fish/seafood': '5aadff',
        'plant':'#7b9f20',
        'plant derivative': '#acae64',
        'dairy':'rgb(250,250,250)',
        'nut/seed/pulse':  '#8D6645'
       }
                   

ingr_color=[colors[ingr_category[v]] for v in V]#list of ingredient(node) colors

edges=Plotly_edges(E, node_position)
nodes=[dict(x=[node_position[idx][0]   for idx in cat_indices[k]],
            y=[node_position[idx][1]   for idx in cat_indices[k]],
            mode='markers',
            name=categories[k],
            marker=dict(symbol='dot',
                        size=9, 
                        color= colors[categories[k]],
                        line=dict(color='rgb(140,140,140)', width=0.5)
                       ),
           text=[V[idx]+'<br>prevalence='+'{:0.5f}'.format(ingr_preval[idx]) for idx in cat_indices[k]],
           hoverinfo='text'
          ) for k in range(n_cat)
     ]



data=[edges]+nodes
layout=layout2d(V, node_position)
fig=dict(data=data, layout=layout)
iplot(fig)

### Flavor category subnetwork

In order to plot the 3d graph of a category define the following  functions:

In [4]:
def category_data(V, Edges, ingr_category, ingr_preval, d_index_ingr, category):
    #prepare data for the subgraph of a category
    #categ is a string
    index_thiscat=categories.index(category)
    
    subg_Edges=[e  for e in Edges if ingr_category[e[0]]==category or ingr_category[e[1]]==category]
    subg_E=[(V.index(e[0]), V.index(e[1])) for e in subg_Edges]# edges as tuples of ints (indices of linked nodes
                                                           #in the list of global nodes)

    subg_nodes=np.array([list(e) for e in subg_E]).reshape(2*len(subg_E))
    subg_nodes=np.unique(subg_nodes)# all nodes  linked with nodes in this category
    nodes_categ=list(set([ingr_category[V[idx]] for idx in subg_nodes]))
    nr_cat=len(nodes_categ)
        
    subg_g=ig.Graph()
    subg_g.add_vertices(subg_nodes.tolist())
    L=len(subg_g.vs)
    ig_nodes=dict(zip(subg_g.vs['name'], range(L)))
    J= [[n  for n in subg_nodes if ingr_category[d_index_ingr[n]]==nodes_categ[k]] for k in range(nr_cat)]
    I=[[ig_nodes[n]  for n in subg_nodes if ingr_category[d_index_ingr[n]]==nodes_categ[k]] for k in range(nr_cat)]
    ig_edges=[(ig_nodes[e[0]], ig_nodes[e[1]]) for e in subg_E]
    subg_g.add_edges(ig_edges)
    subg_layt=np.array(subg_g.layout('kk3d') )

    Xe=[]
    Ye=[]
    Ze=[]
    for e in ig_edges:
        Xe+=[subg_layt[e[0]][0], subg_layt[e[1]][0], None]
        Ye+=[subg_layt[e[0]][1], subg_layt[e[1]][1], None]  
        Ze+=[subg_layt[e[0]][2], subg_layt[e[1]][2], None]
    subg_edges= dict(type='scatter3d', 
                     x=Xe,
                     y=Ye,
                     z=Ze,
                     mode='lines',
                     line=dict(color='rgb(160,160,160)', width=0.75),
                     hoverinfo='none',
                     name='edge'
                 )

    subg_nodes=[dict(type='scatter3d',
                     x=subg_layt[I[k],0],
                     y=subg_layt[I[k],1],
                     z=subg_layt[I[k],2],
                     mode='markers',
                     name=nodes_categ[k],
                     marker=dict(symbol='dot',
                                 size=9, 
                                 color= colors[nodes_categ[k]],
                                 line=dict(color='rgb(140,140,140)', width=0.5)
                                ),
                    text=[V[idx]+'<br>prevalence='+'{:0.5f}'.format(ingr_preval[idx]) for idx in J[k]],
                    hoverinfo='text' 
                   ) for k in range(nr_cat)]
    return [subg_edges]+ subg_nodes

#layout for 3d subnetworks
def layout3d(layout2d, subgraph_title):
    noaxis=dict(showbackground=False,
                showgrid=False,
                showline=False,
                showticklabels=False,
                ticks='',
                title='',
                zeroline=False)
    
    layout_cat=copy.deepcopy(layout2d)
    layout_cat['annotations'][1:]=[]
    layout_cat.update(title=subgraph_title,
                      width=900,
                      height=800,
                      scene=dict(camera = dict(eye=dict(x=1.15, y=1.15, z=1.15)),
                      aspectratio=dict(x=1, y=1, z=0.95),
                      xaxis=dict(noaxis),
                      yaxis=dict(noaxis), 
                      zaxis=dict(noaxis),
               
                        )
                 )
    return layout_cat

In [5]:
category='meat'
meat_data=category_data(V, Edges, ingr_category, ingr_preval, d_index_ingr, category)
meat_title='3D Subgraph of the ingredients in the '+category+' category'
layout_meat=layout3d(layout, meat_title)
fig_meat=dict(data=meat_data, layout=layout_meat)
iplot(fig_meat)




In [ ]:
category='vegetable'
veg_data=category_data(V, Edges, ingr_category, ingr_preval, d_index_ingr, category)
veg_title='3D Subgraph of the ingredients in the '+category+' category'
layout_veg=layout3d(layout, veg_title)
fig_veg=dict(data=veg_data, layout=layout_veg)
iplot(fig_veg)


In [ ]:
category='fish/seafood'
fish_data=category_data(V, Edges, ingr_category, ingr_preval, d_index_ingr, category)
fish_title='3D Subgraph of the ingredients in the '+category+' category'
layout_fish=layout3d(layout, fish_title)
fig_fish=dict(data=fish_data, layout=layout_fish)
iplot(fig_fish)




In [ ]:
category='spice'
spice_data=category_data(V, Edges, ingr_category, ingr_preval, d_index_ingr, category)
spice_title='3D Subgraph of the ingredients in the '+category+' category'
layout_spice=layout3d(layout, spice_title)
fig_spice=dict(data=spice_data, layout=layout_spice)
iplot(fig_spice)


In [6]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()